In [107]:
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import  MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, f1_score,jaccard_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

monster_df = pd.read_csv('../data/processed/Monster_modeling_df.csv')

monster_df.drop(columns={"Unnamed: 0", "Monster Name", "Size",'Type','Alignment'},axis=1, inplace=True)
monster_df.head()



,Traits,Reactions,Armor Class,Hit Points,Challenge,Proficiency Bonus,STR,DEX,CON,INT,...,chaotic good,chaotic neutral,lawful evil,lawful good,lawful neutral,neutral,neutral evil,neutral good,unaligned,total_stats
0,1,0,19,207,15.0,5,23,12,21,18,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,106
1,1,0,19,243,16.0,5,27,10,25,16,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,112
2,1,0,18,200,13.0,5,22,10,22,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86
3,1,0,15,90,5.0,3,14,20,14,6,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,70
4,0,0,12,19,0.5,2,16,14,14,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,69


In [ ]:
#Input Variables: Size, Type, Environment, Alignment, CR(disguised as 4x player level)
#Output Variables: Stats, Saving Throws, Traits(barebones), Attack Bonus, Damage per Round, Reactions (y/n), Legendary Actions (y/n)

LinearRegression (and related)
KNeighborsRegressor
DecisionTreeRegressor
RandomForestRegressor (and related)

In [5]:
monster_df.columns

Index(['Monster Name', 'Size', 'Type', 'Alignment', 'Traits', 'Reactions',
       'Armor Class', 'Hit Points', 'Challenge', 'Proficiency Bonus', 'STR',
       'DEX', 'CON', 'INT', 'WIS', 'CHA', 'Legendary Actions', 'Attack_Bonus',
       'Spell_Bonus', 'Spell_Save_DC', 'WIS_SV', 'INT_SV', 'CHA_SV', 'STR_SV',
       'DEX_SV', 'CON_SV', 'Arctic', 'Coastal', 'Desert', 'Forest',
       'Grassland', 'Hill', 'Mountain', 'NA', 'Swamp', 'Underdark',
       'Underwater', 'Urban', 'Average_Damage_per_Round', 'Damage Resistances',
       'Damage Immunities', 'Condition Immunities', 'Damage Vulnerabilities',
       'Spellcaster', 'Magic Resistance', 'Legendary Resistance',
       'Regeneration', 'Undead Fortitude', 'Pack Tactics', 'Damage Transfer',
       'Angelic Weapons', 'Charge', 'size_ordinal', 'aberration', 'beast',
       'celestial', 'construct', 'dragon', 'elemental', 'fey', 'fiend',
       'giant', 'humanoid', 'monstrosity', 'ooze', 'plant', 'swarm', 'undead',
       'any alignment', 'a

In [127]:
## Train Test Split

X = monster_df[['Challenge','Coastal','Arctic','Desert','Forest','Grassland', 'Hill', 'Mountain', 'NA', 'Swamp', 'Underdark','Underwater', 'Urban','size_ordinal', 'aberration', 'beast',
       'celestial', 'construct', 'dragon', 'elemental', 'fey', 'fiend',
       'giant', 'humanoid', 'monstrosity', 'ooze', 'plant', 'swarm', 'undead',
       'any alignment', 'any chaotic alignment', 'any evil alignment',
       'any non-good alignment', 'any non-lawful alignment', 'chaotic evil',
       'chaotic good', 'chaotic neutral', 'lawful evil', 'lawful good',
       'lawful neutral', 'neutral', 'neutral evil', 'neutral good',
       'unaligned']]
Y = monster_df[['Hit Points','Armor Class','STR','CON','WIS','INT','CHA','STR_SV','DEX_SV','CON_SV','WIS_SV','INT_SV','CHA_SV','Legendary Actions', 'Attack_Bonus','Average_Damage_per_Round', 'Damage Resistances',
       'Damage Immunities', 'Condition Immunities', 'Damage Vulnerabilities',
       'Spellcaster', 'Magic Resistance', 'Legendary Resistance',
       'Regeneration', 'Undead Fortitude', 'Pack Tactics', 'Damage Transfer',
       'Angelic Weapons', 'Charge', 'total_stats']]


x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=42)





## Linear Regression for Multioutput Regression

In [131]:
model = MultiOutputRegressor(LinearRegression())
model.fit(x_train, y_train)
print("Training Score: ", model.score(x_train, y_train))
print("Testing Score: ",model.score(x_test, y_test))
yhat_l = model.predict(x_test)

Training Score:  0.7070818680824811
Testing Score:  0.6008806351547723


## k-Nearest Neighbors for Multioutput Regression

In [132]:
model_k = MultiOutputRegressor(KNeighborsRegressor())
model_k.fit(x_train, y_train)
print("Training Score: ", model_k.score(x_train, y_train))
print("Testing Score: ",model_k.score(x_test, y_test))
yhat_l = model.predict(x_test)

Training Score:  0.6855566230992544
Testing Score:  0.5834105958754812


## Decision Tree Regressor for Multioutput Regression

In [133]:
model_D = MultiOutputRegressor(DecisionTreeRegressor(random_state=0))
model_D.fit(x_train, y_train)
print("Training Score: ", model_D.score(x_train, y_train))
print("Testing Score: ",model_D.score(x_test, y_test))
yhat_l = model.predict(x_test)

Training Score:  0.9887761611262125
Testing Score:  0.4953520957661531


## Random Forest Regressor for Multioutput Regression

In [130]:
model_R = MultiOutputRegressor(RandomForestRegressor(max_depth=7,random_state=0))
model_R.fit(x_train, y_train)
print("Training Score: ", model_R.score(x_train, y_train))
print("Testing Score: ",model_R.score(x_test, y_test))

Training Score:  0.8919863669991325
Testing Score:  0.6706652268979919
